In [39]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np

main_path = Path(r'C:\Users\Richard\Desktop\ABCD_Study\Publication\abcd_paper')
sys.path.append(str(main_path))

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import src.data.preprocess_data as prep
from src.data.data_loader import RepeatedStratifiedKFoldDataloader
import src.data.var_names as abcd_vars

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
total_df = prep.load_complete_df(main_path / 'data' / 'raw')
total_df.head()

,age,female,married,race_ethnicity_Asian,race_ethnicity_Black,race_ethnicity_Hispanic,race_ethnicity_Other,race_ethnicity_White,high.educ_< HS Diploma,high.educ_Bachelor,...,Major Depressive Disorder,Bipolar Disorder,Psychotic Symptoms,ADHD,Oppositional Defiant Disorder,Conduct Disorder,PTSD,Obsessive Compulsive Disorder,Sleep Problems,Suicide Tendency
src_subject_id,,,,,,,,,,,,,,,,,,,,,
NDAR_INV007W6H7B,126,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00CY2MDM,130,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00HEV6HB,124,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00LJVZK2,121,0,0,0,0,0,1,0,1,0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
NDAR_INV00NPMHND,118,1,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [3]:
len(total_df)

8592

In [4]:
total_df.index.equals(total_df.index.str.upper())

True

In [5]:
family_df = pd.read_csv(main_path / 'data' / 'raw' / 'acspsw03.txt', sep='\t', skiprows=(1, 1))
family_df['src_subject_id'] = family_df['src_subject_id'].str.upper()
family_df = family_df[['src_subject_id', 'rel_family_id']]

In [6]:
list(family_df.columns)

['src_subject_id', 'rel_family_id']

In [7]:
family_df

,src_subject_id,rel_family_id
0,NDAR_INV9EVRB30H,106
1,NDAR_INV59BE4FA2,4882
2,NDAR_INVF0C17HWX,11635
3,NDAR_INV6F1LPHLT,6145
4,NDAR_INVZWACXHZ6,10922
...,...,...
11870,NDAR_INVKAPXUG5A,5823
11871,NDAR_INV1NW3HM13,5386
11872,NDAR_INVMTVMYZRH,1548
11873,NDAR_INVZED6PZKP,3576


In [8]:
len(family_df['rel_family_id'].unique())

9987

In [31]:
# Randomly select one child per family
subjects = family_df.groupby(by='rel_family_id').apply(
    lambda x: x.sample(1, random_state=3)
).reset_index(drop=True)['src_subject_id']

In [35]:
new_df = total_df.merge(right=subjects, how='inner', left_index=True, right_on='src_subject_id')
new_df

,age,female,married,race_ethnicity_Asian,race_ethnicity_Black,race_ethnicity_Hispanic,race_ethnicity_Other,race_ethnicity_White,high.educ_< HS Diploma,high.educ_Bachelor,...,Bipolar Disorder,Psychotic Symptoms,ADHD,Oppositional Defiant Disorder,Conduct Disorder,PTSD,Obsessive Compulsive Disorder,Sleep Problems,Suicide Tendency,src_subject_id
3978,126,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NDAR_INV007W6H7B
1939,124,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NDAR_INV00HEV6HB
6920,121,0,0,0,0,0,1,0,1,0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,NDAR_INV00LJVZK2
3446,118,1,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NDAR_INV00NPMHND
7327,114,1,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NDAR_INV00R4TXET
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7844,122,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NDAR_INVZZJ3A7BK
4182,129,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NDAR_INVZZL0VA2F
7845,110,1,1,0,0,0,0,1,0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,NDAR_INVZZLZCKAY
5914,120,1,1,0,0,0,1,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NDAR_INVZZZ2ALR6


In [38]:
new_df.index = new_df['src_subject_id']
new_df = new_df.drop(columns=['src_subject_id'])

In [40]:
single_child_df = prep.select_one_child_per_family(main_path / 'data' / 'raw', total_df, 0)
single_child_df

,age,female,married,race_ethnicity_Asian,race_ethnicity_Black,race_ethnicity_Hispanic,race_ethnicity_Other,race_ethnicity_White,high.educ_< HS Diploma,high.educ_Bachelor,...,Major Depressive Disorder,Bipolar Disorder,Psychotic Symptoms,ADHD,Oppositional Defiant Disorder,Conduct Disorder,PTSD,Obsessive Compulsive Disorder,Sleep Problems,Suicide Tendency
src_subject_id,,,,,,,,,,,,,,,,,,,,,
NDAR_INV007W6H7B,126,0,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
NDAR_INV00LJVZK2,121,0,0,0,0,0,1,0,1,0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
NDAR_INV00NPMHND,118,1,1,0,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
NDAR_INV00R4TXET,114,1,1,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NDAR_INV00U4FTRU,130,1,0,0,0,0,1,0,0,0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NDAR_INVZZJ3A7BK,122,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
NDAR_INVZZL0VA2F,129,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
NDAR_INVZZLZCKAY,110,1,1,0,0,0,0,1,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
